**IMPORTS**

In [ ]:
import datetime
import pandas as pd
from typing import List, Tuple, cast, Union, Dict
from matplotlib import pyplot as plt
from sklearn.multioutput import RegressorChain
from sklearn.linear_model import LinearRegression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.calibration import LabelEncoder
from sklearn.metrics import mean_squared_log_error
from xgboost import XGBRegressor
import optuna
from gsp.utils.utils import (
    make_mw_in_groups,
    make_shift_in_groups,
    ColumnTransformerWrapper,
    show,
    get_nth_previous_working_date,
)
from data import SCRAPED_STOCK_FILE_PATH, save_output, load_output

**FUNCTIONS**

In [ ]:
def root_mean_squared_log_error(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
    return cast(float, mean_squared_log_error(y_true, y_pred) ** 0.5)


def load_data() -> pd.DataFrame:
    return pd.read_csv(
        SCRAPED_STOCK_FILE_PATH,
        dtype={
            "Date": "period[D]",
            "Open": "float",
            "High": "float",
            "Low": "float",
            "Close": "float",
            "Volume": "int",
            "Area": "category",
            "Name": "category",
        },
    )


def clean_data(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy(deep=True)
    periods = pd.date_range(
        start=df["Date"].min().to_timestamp().date(), end=df["Date"].max().to_timestamp().date(), freq="B"
    )
    periods_df = pd.DataFrame({"Date": periods}, dtype="period[D]")

    clean_data = (
        cast(
            pd.DataFrame,
            periods_df.set_index("Date")
            .join(df.set_index("Date"))
            .set_index("Name", append=True)
            .sort_index()
            .unstack("Name")
            .ffill()
            .stack("Name", future_stack=True),  # type: ignore
        )
        .reset_index()
        .dropna(subset=["Name"])
        .set_index(["Date", "Name"])
    )

    return clean_data


def engineer_features(
    df: pd.DataFrame,
    categorical_features: List[str] = [],
    shift_list: List[int] = [],
    mwm_list: List[int] = [],
    label_features: List[str] = [],
) -> pd.DataFrame:

    df = df.copy()

    cat_features_to_use: List[str] = [*categorical_features, *label_features]

    if "DayOfWeek" in cat_features_to_use:
        df["DayOfWeek"] = cast(pd.PeriodIndex, df.index.get_level_values("Date")).dayofweek

    if "Month" in cat_features_to_use:
        df["Month"] = cast(pd.PeriodIndex, df.index.get_level_values("Date")).month

    if "Year" in cat_features_to_use:
        df["Year"] = cast(pd.PeriodIndex, df.index.get_level_values("Date")).year

    if "WeekOfYear" in cat_features_to_use:
        df["WeekOfYear"] = cast(pd.PeriodIndex, df.index.get_level_values("Date")).week  # type: ignore

    if "DayOfMonth" in cat_features_to_use:
        df["DayOfMonth"] = cast(pd.PeriodIndex, df.index.get_level_values("Date")).day

    if "Quarter" in cat_features_to_use:
        df["Quarter"] = cast(pd.PeriodIndex, df.index.get_level_values("Date")).quarter

    if "AreaCat" in cat_features_to_use:
        df["AreaCat"] = df["Area"]

    grouped_lags: List[pd.DataFrame | pd.Series] = [
        make_shift_in_groups(df, groupby=["Name"], column="Close", shift=shift_list),
        make_mw_in_groups(df, groupby=["Name"], column="Close", window=mwm_list),
    ]

    df_grouped_lags = df.join(grouped_lags, how="left")

    return df_grouped_lags


def process_data(
    df: pd.DataFrame,
    n_steps: int,
    categorical_features: List[str],
    label_features: List[str] = [],
    days_back_to_consider: int | None = None,
) -> Tuple[pd.DataFrame, pd.DataFrame]:

    df = df.copy()

    """First date where all stocks have data"""
    first_all_valid_date: datetime.date = (
        cast(pd.Period, df.unstack("Name").dropna().first_valid_index()).to_timestamp().date()
    )

    latest_date: datetime.date = cast(pd.Period, df.index.get_level_values("Date").max()).to_timestamp().date()

    earliest_date: datetime.date = first_all_valid_date

    starting_date_to_consider: datetime.date | None = None

    if days_back_to_consider is not None:
        starting_date_to_consider = get_nth_previous_working_date(n=days_back_to_consider, date=latest_date)

    if starting_date_to_consider is not None and starting_date_to_consider < first_all_valid_date:
        show(
            f"WARNING: starting_date_to_consider ({starting_date_to_consider}) is before the first date where all stocks have data. Using {first_all_valid_date} instead."
        )
    elif starting_date_to_consider is not None:
        earliest_date = starting_date_to_consider

    show(f"Earliest date: {earliest_date.isoformat()}")

    ctw = ColumnTransformerWrapper(
        transformers=[
            ("onehot", OneHotEncoder(drop="first"), categorical_features),
            ("label", LabelEncoder(), label_features),
        ],
        remainder="passthrough",
    )

    df_from_earliest = df.loc[earliest_date.isoformat() :]
    X = ctw.fit_transform(df_from_earliest.drop(columns=["Adj Close", "Volume", "High", "Low", "Open", "Area"]))
    y = make_shift_in_groups(df, groupby=["Name"], column="Close", shift=[-i for i in range(1, n_steps + 1)])

    y, X = y.align(X.dropna(), axis=0, join="inner")

    return X, y


def split_data(
    X: pd.DataFrame,
    y: pd.DataFrame,
    n_steps: int,
) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """A function that splits the time series data into train and test sets in regards
    to the number of forecasting steps. It will test the model on the last 2*n_steps.

    Args:
        X (pd.DataFrame): Features dataframe
        y (pd.DataFrame): Target dataframe
        n_steps (int): Number of steps of the forecasting task

    Returns:
        Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame]: X_train, y_train, X_test, y_test
    """
    latest_date = cast(pd.Period, X.index.get_level_values("Date").max()).to_timestamp().date()
    test_end_date = get_nth_previous_working_date(n=n_steps, date=latest_date)
    test_start_date = get_nth_previous_working_date(n=2 * n_steps - 1, date=latest_date)
    train_end_date = get_nth_previous_working_date(n=2 * n_steps, date=latest_date)

    X_train = X.loc[: train_end_date.isoformat()]
    y_train = y.loc[: train_end_date.isoformat()]
    X_test = X.loc[test_start_date.isoformat() : test_end_date.isoformat()]
    y_test = y.loc[test_start_date.isoformat() : test_end_date.isoformat()]

    return X_train, y_train, X_test, y_test


def create_model(**hyper_params):
    model = RegressorChain(XGBRegressor(**hyper_params))

    return model


def fit_and_predict(
    model: Union[RegressorChain, MultiOutputRegressor, LinearRegression],
    X_train: pd.DataFrame,
    y_train: pd.DataFrame,
    X_query: pd.DataFrame,
):
    model = model.fit(X_train, y_train)
    y_pred = pd.DataFrame(model.predict(X_query), index=X_query.index, columns=y_train.columns)  # type: ignore

    return y_pred


def convert_last_prediction_to_output(df: pd.DataFrame) -> pd.DataFrame:

    df = df.copy()

    latest_date = cast(pd.Period, df.index.get_level_values("Date").max()).to_timestamp().date()

    y_latest = cast(pd.DataFrame, df.loc[latest_date.isoformat()]).rename(
        columns={
            f"Close_lead_{i}": get_nth_previous_working_date(n=-i, date=latest_date).isoformat()
            for i in range(1, len(df.columns) + 1)
        }
    )

    y_output = (
        y_latest.reset_index()
        .melt(id_vars=["Name"], var_name="Date", value_name="Close")
        .astype(
            {
                "Date": "period[D]",
                "Name": "category",
                "Close": "float",
            }
        )
        .set_index(["Date", "Name"])
    )

    return y_output


def optimize_with_optuna(
    X_train: pd.DataFrame, y_train: pd.DataFrame, X_test: pd.DataFrame, y_test: pd.DataFrame, n_optimize_trials: int
) -> Tuple[Dict, float]:
    study = optuna.create_study(direction="minimize", study_name="xgboost_optuna")

    def optuna_optimize_objective(trial: optuna.Trial) -> float:
        params = {
            "n_estimators": trial.suggest_int("n_estimators", 100, 1200, step=50),  # Number of trees in the ensemble
            "max_depth": trial.suggest_int("max_depth", 3, 15),  # Maximum depth of each tree
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),  # Learning rate
            "subsample": trial.suggest_float("subsample", 0.5, 1.0),  # Subsample ratio of the training instances
            "colsample_bytree": trial.suggest_float(
                "colsample_bytree", 0.5, 1.0
            ),  # Subsample ratio of columns when constructing each tree
            "gamma": trial.suggest_float(
                "gamma", 0.01, 10.0, log=True
            ),  # Minimum loss reduction required to make a further partition on a leaf node of the tree
            "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 100.0, log=True),  # L1 regularization term on weights
            "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 100.0, log=True),  # L2 regularization term on weights
            "min_child_weight": trial.suggest_float(
                "min_child_weight", 1, 100, log=True
            ),  # Minimum sum of instance weight (hessian) needed in a child
        }

        model = create_model(**params)
        y_pred = fit_and_predict(model, X_train, y_train, X_test)
        rmsle = root_mean_squared_log_error(y_test, y_pred)

        return rmsle

    study.optimize(optuna_optimize_objective, n_trials=n_optimize_trials)
    best_params = study.best_params
    best_score = study.best_value

    return best_params, best_score


def execute_test_run(
    n_steps: int,
    days_back_to_consider: int,
    categorical_features: List[str],
    label_features: List[str],
    shift_list: List[int],
    mwm_list: List[int],
    hyper_params: Dict = {},
    n_optimize_trials: int = -1,
) -> Tuple[pd.DataFrame, pd.DataFrame, float, Dict, datetime.date]:

    stocks = load_data()
    clean = clean_data(stocks)
    features = engineer_features(
        clean, categorical_features, shift_list=shift_list, mwm_list=mwm_list, label_features=label_features
    )
    X, y = process_data(features, n_steps, categorical_features, days_back_to_consider=days_back_to_consider)
    X_train, y_train, X_test, y_test = split_data(X, y, n_steps)
    test_prediction_date = cast(pd.Period, X_test.index.get_level_values("Date").max()).to_timestamp().date()
    show(X.columns)
    show(y.columns)
    # --- NOTICE ---
    # Default run with given hyperparameters
    model_default = create_model(**hyper_params)
    y_test_default_pred = fit_and_predict(model_default, X_train, y_train, X_test)
    rmsle_default = root_mean_squared_log_error(y_test, y_test_default_pred)

    best_params = hyper_params
    best_score = rmsle_default

    if n_optimize_trials > 0:
        opt_params, opt_score = optimize_with_optuna(X_train, y_train, X_test, y_test, n_optimize_trials)

        if opt_score < best_score:
            best_params = opt_params
            best_score = opt_score

    show(f"Best parameters: {best_params}")
    show(f"Best score: {best_score}")

    # --- NOTICE ---
    # Final run with best hyperparameters
    opt_model = create_model(**best_params)
    y_test_pred = fit_and_predict(opt_model, X_train, y_train, X_test)
    rmsle = root_mean_squared_log_error(y_test, y_test_pred)
    y_test_output = convert_last_prediction_to_output(y_test_pred)

    show(f"Root Mean Squared Log Error: {rmsle}")

    return features, y_test_output, rmsle, best_params, test_prediction_date


def execute_real_run(
    n_steps: int,
    days_back_to_consider: int,
    categorical_features: List[str],
    label_features: List[str],
    shift_list: List[int],
    mwm_list: List[int],
    hyper_params: Dict = {},
) -> Tuple[pd.DataFrame, datetime.date]:

    stocks = load_data()
    clean = clean_data(stocks)
    features = engineer_features(
        clean, categorical_features, label_features=label_features, shift_list=shift_list, mwm_list=mwm_list
    )
    X, y = process_data(features, n_steps, categorical_features, days_back_to_consider=days_back_to_consider)
    X_train, y_train = X.align(y.dropna(), axis=0, join="inner")

    prediction_date = cast(pd.Period, X.index.get_level_values("Date").max()).to_timestamp().date()
    X_query = X.loc[prediction_date.isoformat() :]

    model = create_model(**hyper_params)
    y_pred = fit_and_predict(model, X_train, y_train, X_query)
    y_output = convert_last_prediction_to_output(y_pred)

    return y_output, prediction_date

**FORECASTING PROBLEM DEFINITION**

In [ ]:
n_steps: int = 15
n_optimize_trials: int = 50  # --- NOTICE --- Negative value means no optimization
days_back_to_consider: int = 5 * 252
label_features: List[str] = ["Year"]
categorical_features: List[str] = ["DayOfWeek", "AreaCat"]
shift_list: List[int] = [1, 2, 3]
mwm_list: List[int] = [5, 10, 15]

**TEST EXECUTION**

In [ ]:
features, y_test_output, rmsle, opt_hyper_params, test_prediction_date = execute_test_run(
    n_steps=n_steps,
    days_back_to_consider=days_back_to_consider,
    categorical_features=categorical_features,
    label_features=label_features,
    shift_list=shift_list,
    mwm_list=mwm_list,
    n_optimize_trials=n_optimize_trials,
)

test_history_log_df = load_output("test_history_log.csv")
current_input_df = pd.DataFrame(
    {
        "Date": [test_prediction_date],
        "RMSLE": [rmsle],
        "HyperParams": [opt_hyper_params],
        "CategoricalFeatures": [categorical_features],
        "LabelFeatures": [label_features],
        "NSteps": [n_steps],
        "Shift_list": [shift_list],
        "MWM_list": [mwm_list],
        "Version": [len(test_history_log_df) + 1],
        "Phase": [1],
        "DaysBackToConsider": [days_back_to_consider],
    }
).set_index(["Version"])

test_history_log_df = pd.concat([test_history_log_df, current_input_df], axis=0)

save_output(test_history_log_df, "test_history_log.csv")

In [ ]:
diplay_last_n_days = 150
unique_names = y_test_output.index.get_level_values("Name").unique()
for name in unique_names:
    fig = plt.figure(figsize=(10, 5))
    fig.suptitle(name)

    cast(pd.DataFrame, features[features.index.get_level_values("Name") == name]).reset_index("Name")["Close"].tail(
        diplay_last_n_days
    ).plot(label=f"Close for {name}", ax=fig.gca())

    cast(pd.DataFrame, y_test_output[y_test_output.index.get_level_values("Name") == name]).reset_index("Name")[
        "Close"
    ].plot(label=f"Predicted Close for {name}", ax=fig.gca())

    plt.show()

**REAL EXECUTION**

In [ ]:
y_real_output, prediction_date = execute_real_run(
    n_steps=n_steps,
    days_back_to_consider=days_back_to_consider,
    categorical_features=categorical_features,
    label_features=label_features,
    shift_list=shift_list,
    mwm_list=mwm_list,
    hyper_params=opt_hyper_params,
)

save_output(y_real_output, f"{prediction_date.isoformat()}_prediction_{n_steps}_steps.csv")